In [1]:
import os

list_file = sorted(list(os.walk('./file/1.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/1.url/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：550


['https://ecat.spectrapremium.com/en/parts/BMP02A',
 'https://ecat.spectrapremium.com/en/parts/BMP03A',
 'https://ecat.spectrapremium.com/en/parts/BMP05A',
 'https://ecat.spectrapremium.com/en/parts/BMP06A',
 'https://ecat.spectrapremium.com/en/parts/BMP08A',
 'https://ecat.spectrapremium.com/en/parts/BMP09A',
 'https://ecat.spectrapremium.com/en/parts/BMP10A',
 'https://ecat.spectrapremium.com/en/parts/BMP11A',
 'https://ecat.spectrapremium.com/en/parts/BMP13A',
 'https://ecat.spectrapremium.com/en/parts/BMP14A',
 'https://ecat.spectrapremium.com/en/parts/BMP15A',
 'https://ecat.spectrapremium.com/en/parts/BMP17A',
 'https://ecat.spectrapremium.com/en/parts/BMP19A',
 'https://ecat.spectrapremium.com/en/parts/BMP20A',
 'https://ecat.spectrapremium.com/en/parts/BMP21A',
 'https://ecat.spectrapremium.com/en/parts/BMP22A',
 'https://ecat.spectrapremium.com/en/parts/BMP23A',
 'https://ecat.spectrapremium.com/en/parts/BMP24A',
 'https://ecat.spectrapremium.com/en/parts/CATP04A',
 'https://e

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = input_.loc[a, 'Url']
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            market = html.xpath('//div[@class="part__canada-ligne"]/img/@title')[0].strip() if html.xpath('//div[@class="part__canada-ligne"]/img') else ''
            with open('./part.html', 'w', encoding='UTF-8') as file:
                file.write(str(soup))
            break
    except KeyboardInterrupt:
        break
    except:
        continue

market

1


''

In [8]:
type_ = ''.join([text.strip() for text in html.xpath('//div[@class="part__canada-ligne"]/text()')]).strip()

type_

'Oil Pan'

In [9]:
dict_src, list_src = {}, [f'https://ecat.spectrapremium.com{src.strip()}' for src in html.xpath('//a[@class="part__magic-scroll-images__thumbnail"]/@href')]
while 'https://ecat.spectrapremium.com#' in list_src:
    list_src.remove('https://ecat.spectrapremium.com#')

for i, src in enumerate(list_src):
    dict_src[str(i)] = src

dict_src

{'0': 'https://ecat.spectrapremium.com/img/produits/BMP02A_TOP_P04.jpg',
 '1': 'https://ecat.spectrapremium.com/img/produits/BMP02A_BAC_P01.jpg',
 '2': 'https://ecat.spectrapremium.com/img/produits/BMP02A_FRO_P01.jpg',
 '3': 'https://ecat.spectrapremium.com/img/produits/BMP02A_BOT_P01.jpg'}

In [10]:
part_number = html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip()

part_number

'BMP02A'

In [11]:
list_div, dict_count = html.xpath('//dl[@class="part__specifications"]/div[@class="part__nom-spec-valeur-spec"]'), {}
for div in list_div:
    key = div.xpath('./dt[@class="part__nom-spec"]/text()')[0].strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_specification, dict_no = {}, {}
for i, div in enumerate(list_div):
    key = div.xpath('./dt[@class="part__nom-spec"]/text()')[0].strip()
    if dict_count[key] == 1:
        dict_specification[str(i)] = {key: div.xpath('./dd[@class="part__valeur-spec"]/text()')[0].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_specification[str(i)] = {f'''{key}-{dict_no[key]}''': div.xpath('./dd[@class="part__valeur-spec"]/text()')[0].strip()}

dict_specification

{'0': {'Material': 'Aluminum'},
 '1': {'Height': '3.25 in'},
 '2': {'Drain Plug Thread': 'M12 X 1.25'},
 '3': {'Liter': '5'},
 '4': {'Pop. Code': 'D'}}

In [12]:
vehicle_page = int(html.xpath('//li[@class="paginator__last"]/a/@href')[0].strip().removesuffix('#applications').split('=')[-1]) if html.xpath('//li[@class="paginator__last"]') else 1

vehicle_page

1

In [13]:
import json

df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Market': market,
                         'Type': type_,
                         'Part Number': part_number,
                         'Vehicle': '',
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': json.dumps(dict_src),
                         'Json_Specification': json.dumps(dict_specification),
                         'Vehicle Page': vehicle_page}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Market,Type,Part Number,Picture,Url,Json_Src,Json_Specification,Vehicle Page
0,1,,Oil Pan,BMP02A,,https://ecat.spectrapremium.com/en/parts/BMP02A,"{""0"": ""https://ecat.spectrapremium.com/img/pro...","{""0"": {""Material"": ""Aluminum""}, ""1"": {""Height""...",1


In [14]:
crawler_status = 'ok'

crawler_status

'ok'

In [15]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [16]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Market,Type,Part Number,Picture,Url,Json_Src,Json_Specification,Vehicle Page
0,1,,Oil Pan,BMP02A,,https://ecat.spectrapremium.com/en/parts/BMP02A,"{""0"": ""https://ecat.spectrapremium.com/img/pro...","{""0"": {""Material"": ""Aluminum""}, ""1"": {""Height""...",1


In [17]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
